In [1]:
from filler import fill_redacted_text, evaluate_redacted
import json
import tqdm
import time
import pandas as pd

In [2]:
data = pd.read_csv("inputs/radiology.csv")

data.head()
print(len(data))

1000


In [18]:
notes = data["text"]
notes.head()

0    CT CHEST WITHOUT CONTRAST\n\nREASON FOR EXAM: ...
1    MEDICAL HISTORY:  ___ woman with hemolytic ane...
2    INDICATION:  ___ year old man with metastatic ...
3    INDICATION:  ___ man with right-sided testicul...
4    EXAMINATION:  CT LOWER EXT W/C LEFT\n\nINDICAT...
Name: text, dtype: object

In [19]:
notes = notes[:10]
notes

0    CT CHEST WITHOUT CONTRAST\n\nREASON FOR EXAM: ...
1    MEDICAL HISTORY:  ___ woman with hemolytic ane...
2    INDICATION:  ___ year old man with metastatic ...
3    INDICATION:  ___ man with right-sided testicul...
4    EXAMINATION:  CT LOWER EXT W/C LEFT\n\nINDICAT...
5    EXAMINATION:  LIVER OR GALLBLADDER US (SINGLE ...
6    HISTORY:  Ankle pain.\n\nTECHNIQUE:  3 views r...
7    INDICATION:  ___ female with tachypnea.\n\nTEC...
8    INDICATION:  Tripped and fell with left mid fo...
9    HISTORY:  Central line placement.\n\n___.\n\nF...
Name: text, dtype: object

In [20]:
filled_data = []
total = 0

for note in tqdm.tqdm(notes):
    retries = 0
    while retries < 3:
        try:
            filled_text, pii_mappings, filled_values = fill_redacted_text(note)
            evaluate_report = evaluate_redacted(filled_text)  
            
            if evaluate_report["passed"]:
                    
                filled_data.append({
                    "original_text": note,
                    "filled_text": filled_text,
                    "pii_mappings": pii_mappings,
                    "filled_values": filled_values
                })
                
                total += 1
                break
            else:
                print(f"Failed to fill the note: {note}")
                print(evaluate_report["reasoning"])
                time.sleep(1)
        except Exception as e:
            retries += 1
            print(f"Error: {e}")
            time.sleep(1)

json.dump(filled_data, open("outputs/radiology_filled.json", "w"))
print(f"Generated {total} filled notes")

 50%|█████     | 5/10 [00:01<00:01,  3.33it/s]

Error: replace() argument 2 must be str, not int


100%|██████████| 10/10 [00:04<00:00,  2.04it/s]

Generated 10 filled notes
